## Import libraries and dependencies


In [7]:

import optimiseprime.data_prep as dp
import optimiseprime.data_analysis as da

import datetime as dt

import pandas as pd
import yfinance as yf
import quantstats as qs
qs.extend_pandas()

## Request user for portfolio details, and fetch price data from yfinance Yahoo! Finance API

In [8]:
portfolio_choice = ""
while portfolio_choice not in [1, 2]:
    try:
        portfolio_choice = int(input(         
            "To analyse an existing portfolio, type 1\n"
            "To analyse a hypothetical portfolio - type 2\n"
        )
                              )
    except Exception:
        print('Error: Invalid Choice.\n')
        
# Get portfolio data from user depending on choice:

ticker_list = []

while len(ticker_list) == 0:
    if portfolio_choice == 1:
        existing_portfolio = dp.get_existing_portfolio()
        ticker_list = []
        for key, value in existing_portfolio.items():
            ticker_list.append(key)
        ticker_list = pd.DataFrame(columns=ticker_list).add_suffix('-USD').columns.tolist()
    elif portfolio_choice == 2:
        ticker_list = dp.get_hypothetical_portfolio()
        # Request investment amount from user
        portfolio_value = dp.get_investment_amt()
    if len(ticker_list) == 0:
        print("You have not entered any tickers.")


# Fetch data from yfinance for each ticker, and create pandas dataframe
portfolio_df = dp.get_ticker_data(ticker_list)
portfolio_df.dropna(inplace = True)

# Keep only tickers in ticker_list for which data is available
ticker_list = [ticker for ticker in list(portfolio_df.columns.levels[0])]


# Print portfolio data for visual confirmation
if portfolio_choice == 1: 
    print(f"--------------------------")                      
    print(f"Existing Portfolio:")
    portfolio_value = int()
    for ticker, units in existing_portfolio.items():
        value = portfolio_df[f"{ticker}-USD"].iloc[-1, 3] * units
        print(f"Value of {units} {ticker}: ${value:.2f}")
        total_value += value
    print(f"\nTotal portfolio value: ${portfolio_value:.2f}\n")       
elif portfolio_choice == 2:
    print(f"--------------------------")                      
    print(f"Hypothetical Portfolio:")
    print(f"{[ticker.replace('-USD', '') for ticker in ticker_list]}")                   
    print(f"Investment amount:")
    print(f"${portfolio_value:.2f}\n")

print(
    f"To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.\n"
    f"Earliest date for which price data is available for all cryptocurrencies in your portfolio: {dt.datetime.date(portfolio_df.index[0])}"
)
print(f"--------------------------")
    

To analyse an existing portfolio, type 1
To analyse a hypothetical portfolio - type 2
 2


Please enter the tickers of your cryptocurrencies one by one
Type 'done' when finished.


Ticker:  eth
Ticker:  btc
Ticker:  xrp
Ticker:  xdc
Ticker:  done
How much do you wish to invest in total?
(Please input amount without currency symbol)
 30000


--------------------------
Hypothetical Portfolio:
['BTC', 'ETH', 'XDC', 'XRP']
Investment amount:
$30000.00

To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.
Earliest date for which price data is available for all cryptocurrencies in your portfolio: 2019-01-12
--------------------------


In [9]:
portfolio_df.tail()


ETH-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2022-01-06  3549.708984  3549.708984  3335.376709  3418.408203  24293791313   
2022-01-07  3417.837891  3420.461914  3117.381348  3193.210449  24494179209   
2022-01-08  3193.502441  3240.304443  3020.880859  3091.972656  16037732700   
2022-01-09  3091.696289  3206.790771  3075.911377  3157.751465  12334683863   
2022-01-10  3157.667480  3177.213379  2949.450195  3077.357910  19629492224   

                              BTC-USD                              \
           daily_return          open          high           low   
Date                                                                
2022-01-06    -0.037173  43565.511719  43748.718750  42645.539062   
2022-01-07    -0.065878  43153.570312  43153.570312  41077.445312   
2022-01-08    -0.031704  41561.464844  42228.941406  40672.277344   
2022-01-09     0.021274  41734.726562  42663.949219  41338.160156   
2022-01-10    -0.025459  41951.582031  42198.058594  39816.937500   

                          ...   XRP-USD                                     \
                   close  ...       low     close      volume daily_return   
Date                      ...                                                
2022-01-06  43160.929688  ...  0.744292  0.781346  2165022786     0.009024   
2022-01-07  41557.902344  ...  0.734617  0.763074  2329624598    -0.023385   
2022-01-08  41733.941406  ...  0.727803  0.747688  1792091501    -0.020163   
2022-01-09  41911.601562  ...  0.741534  0.754505  1182503166     0.009117   
2022-01-10  41718.863281  ...  0.705479  0.735199  1980774272    -0.025587   

             XDC-USD                                                      
                open      high       low     close   volume daily_return  
Date                                                                      
2022-01-06  0.094158  0.094417  0.090549  0.092281  8027778    -0.019862  
2022-01-07  0.092271  0.092310  0.086859  0.086859  7796199    -0.058755  
2022-01-08  0.086866  0.088330  0.082103  0.083637  8327918    -0.037095  
2022-01-09  0.083642  0.088107  0.083002  0.087459  6420355     0.045697  
2022-01-10  0.087247  0.087676  0.081798  0.083591  6638822    -0.044222  

[5 rows x 24 columns]

## Data Analyses

### Calculate ratios:
* Sharpe ratio
* Sortino ratio
* Adjusted sortino ratio
* Gain to Pain ratio

In [10]:
# Calculate each of the following risk-reward ratio types
sharpe = da.calculate_sharpe_ratio(ticker_list, portfolio_df)
sortino =  da.calculate_sortino_ratio(ticker_list, portfolio_df)
adjusted_sortino = da.calculate_adjusted_sortino(ticker_list, portfolio_df)
gain_pain_ratio = da.calculate_gain_pain_ratio(ticker_list, portfolio_df)


# Store all ratios into a dict
ratios_df = pd.DataFrame(
    {
    'sharpe': sharpe,
    'sortino': sortino,
    'adj_sortino': adjusted_sortino,
    'gain_pain': gain_pain_ratio,
    }
)

# Calculate proportion scores for each risk-reward metric

weights = da.calculate_weights(ratios_df)

weights

,sharpe,sortino,adj_sortino,gain_pain
BTC-USD,0.263610,0.239021,0.239021,0.256561
ETH-USD,0.288410,0.259361,0.259359,0.273371
XDC-USD,0.305801,0.360768,0.360773,0.319965
XRP-USD,0.142179,0.140849,0.140846,0.150103


In [11]:
print(
    f"Portfolio allocation recommendations\n"
    f"Based on historical returns from {dt.datetime.date(portfolio_df.index[0])} to {dt.datetime.date(portfolio_df.index[-1])}"
)
print(f"Total portfolio value: ${portfolio_value:.2f}")
print(f"============================================================="
)

# Present all ratios in descending order

for column in ratios_df:
    if column == 'sharpe':
        da.sharpe_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'sortino':
        da.sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'adj_sortino':
        da.adj_sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'gain_pain':
        da.gain_pain_portfolio(ratios_df, weights, portfolio_value)
        

    




Portfolio allocation recommendations
Based on historical returns from 2019-01-12 to 2022-01-10
Total portfolio value: $30000.00

Metric: Sharpe Ratio
--------------
XDC-USD: 1.71
Recommended percentage of total portfolio: 30.58%
Recommended value allocation = $9174.03

ETH-USD: 1.62
Recommended percentage of total portfolio: 28.84%
Recommended value allocation = $8652.31

BTC-USD: 1.48
Recommended percentage of total portfolio: 26.36%
Recommended value allocation = $7908.30

XRP-USD: 0.80
Recommended percentage of total portfolio: 14.22%
Recommended value allocation = $4265.36


Metric: Sortino Ratio
--------------
XDC-USD: 3.32
Recommended percentage of total portfolio: 30.58%
Recommended value allocation = $9174.03

ETH-USD: 2.39
Recommended percentage of total portfolio: 28.84%
Recommended value allocation = $8652.31

BTC-USD: 2.20
Recommended percentage of total portfolio: 26.36%
Recommended value allocation = $7908.30

XRP-USD: 1.30
Recommended percentage of total portfolio: 14.22